In [3]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [4]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [27]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,data_source2,other_projects,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    goals = ['f1','g','recall','precision','pf']
    for goal in goals:
        df = pd.read_csv(data_source1 + '/bellwether_' + goal + '_1.csv')
        _cluster = {}
        for j in range(df.shape[0]):
            cluster_id = df.iloc[j,1]
            _bellwether = [df.iloc[j,5]]
            _cluster[cluster_id] = [_bellwether,list(df_train.iloc[cluster_tree[cluster_id].data_points].index)]       
        bell = birch_bellwether.bellwether(path,df)
        for _bell_cluster in _cluster:
            final_score = bell.bellwether(_cluster[_bell_cluster][0],_cluster[_bell_cluster][1])
            _results = {}
            for s_project in final_score.keys():
                if s_project not in _results.keys():
                    _results[s_project] = {}
                for d_projects in final_score[s_project].keys():
                    if goal == 'g':
                        _goal = 'g-score'
                    else:
                        _goal = goal
                    _results[s_project][d_projects] = np.median(final_score[s_project][d_projects][_goal])
            data_path = data_source2 + '/' + str(_bell_cluster)
            if not Path(data_path).is_dir():
                os.makedirs(Path(data_path))
            _df_results = pd.DataFrame.from_dict(_results, orient='index')
            _df_results = _df_results.to_csv(data_path +'/bellwether_' + goal +'.csv')
            _df_results = pd.read_csv(data_path +'/bellwether_' + goal +'.csv')
        

In [28]:
for i in range(2):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    data_source2 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/1/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = path + '/'
    else:
        _dir = path + '\\'

    projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    #print(projects)
    find_bellwether(data_source1,data_source2,projects,path,i)
    

0
kml.csv
frogatto.csv
kolmafia.csv
kaffeine.csv
pcsx2.csv
clamav.csv
kdenlive.csv
silvertree.csv
simplewebservices.csv
hydrogen.csv
sevenupdate.csv
irrlicht.csv
jfreereport.csv
galaxium.csv
retromenu.csv
taokgame.csv
avisynth2.csv
fbmanager.csv
jchassis.csv
cme.csv
1
patterntesting.csv
magicwars.csv
lphant.csv
prelude.csv
sumatrapdf.csv
google-caja.csv
cleartk.csv
columba.csv
owlib.csv
gss.csv
enlightenment.csv
magicwars.csv
inkscape.csv
google-collections.csv
nh3d.csv
genoviz.csv
bigdata.csv
k3d.csv
mvdsv.csv
biodwh.csv
fluidium.csv
jchassis.csv
fido.csv
empyrean.csv
xined.csv
